# Mask tSNR images

K.Garner, 2021
Note: run by participant, and by session (if unequal n runs between TRs)

In [17]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
import pandas as pd
import os, re, json


In [18]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
thrsh = pe.Workflow(name="msktSNR") # workflow to run the analysis

## Data grabbing and sinking

In [19]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['ts','msks']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['sub', 'TR']],
                           'msks': [['sub', 'TR']]}
dg.inputs.field_template = {'msks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-%s/TR%s/MASKS/*.nii.gz',
                            'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-%s/TR%s/SNR/*/*.nii.gz'}

In [20]:
# # debugging
dg.inputs.sub = '03'
dg.inputs.TR = '1920'
res = dg.run()
res.outputs

210704-08:53:28,821 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmp7cuxlcnk/data-grabber".
210704-08:53:28,826 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210704-08:53:28,832 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



msks = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rCN_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rFEF_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rGPe_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rGPi_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rIPS_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rLOC_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rPut_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rSTN_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rVS_trans.nii.gz']
ts = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz']

In [21]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [22]:
printSubPath(res.outputs.msks)

'sub-03/TR1920'

In [23]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_thrsh-t([0-9]*)', ''),
                 ('_absthat([0-9]*)', ''),
                 ('_mask-t([0-9])*', ''),
                 ('_multi([0-9])*', '')] 
ds.inputs.regexp_substitutions = substitutions

In [24]:
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
#datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
datinf.iterables = [('sub', ['03']), ('TR', ['1920'])]
datinf.iterables 

[('sub', ['03']), ('TR', ['1920'])]

## functions for list cats

In [25]:
def tcat(fullFileList, nMasks=9):
    #return [fullFileList]*nMasks
    return fullFileList*nMasks # use this line when n fullFileList > 1

In [26]:
tcat(res.outputs.ts)

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz',

In [27]:
def maskcat(fullFileList, nCon=2):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

In [28]:
maskcat(res.outputs.msks)

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rCN_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rCN_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rFEF_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rFEF_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rGPe_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rGPe_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rGPi_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rGPi_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rIPS_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rIPS_trans.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rLOC_trans.nii.gz',
 '/scratch/qbi/uqkgarn1

## define output names for t x msk

In [29]:
# warning! needs to be adjusted for each participant
tnms = ['tsnr_ses-02', 'tsnr_ses-03']
msks = ['rCN', 'rFEF', 'rGPe', 'rGPi', 'rIPS', 'rLOC', 'rPut', 'rSTN', 'rVS']
touts = [i+'_'+n+'.nii.gz' for i in msks for n in tnms]

In [30]:
touts

['rCN_tsnr_ses-02.nii.gz',
 'rCN_tsnr_ses-03.nii.gz',
 'rFEF_tsnr_ses-02.nii.gz',
 'rFEF_tsnr_ses-03.nii.gz',
 'rGPe_tsnr_ses-02.nii.gz',
 'rGPe_tsnr_ses-03.nii.gz',
 'rGPi_tsnr_ses-02.nii.gz',
 'rGPi_tsnr_ses-03.nii.gz',
 'rIPS_tsnr_ses-02.nii.gz',
 'rIPS_tsnr_ses-03.nii.gz',
 'rLOC_tsnr_ses-02.nii.gz',
 'rLOC_tsnr_ses-03.nii.gz',
 'rPut_tsnr_ses-02.nii.gz',
 'rPut_tsnr_ses-03.nii.gz',
 'rSTN_tsnr_ses-02.nii.gz',
 'rSTN_tsnr_ses-03.nii.gz',
 'rVS_tsnr_ses-02.nii.gz',
 'rVS_tsnr_ses-03.nii.gz']

## FSL functionality to apply anatomical mask

In [31]:
maskt = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'operand_file', 'out_file'], name='mask-t')
maskt.inputs.operation = 'mul'
maskt.inputs.nan2zeros = True
maskt.inputs.out_file = touts

## Connect workflow

In [32]:
thrsh.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('msks', printSubPath), 'container')]),
               (dg, maskt, [(('ts', tcat), 'in_file')]), 
               (dg, maskt, [(('msks', maskcat), 'operand_file')]),
               (maskt, ds, [('out_file', 'MSKD')])
])
res = thrsh.run()

210704-08:54:29,943 nipype.workflow INFO:
	 Workflow msktSNR settings: ['check', 'execution', 'logging', 'monitoring']
210704-08:54:29,955 nipype.workflow INFO:
	 Running serially.
210704-08:54:29,956 nipype.workflow INFO:
	 [Node] Setting-up "msktSNR.data-grabber" in "/tmp/tmp7cuxlcnk/data-grabber".
210704-08:54:29,962 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210704-08:54:29,967 nipype.workflow INFO:
	 [Node] Finished "msktSNR.data-grabber".
210704-08:54:29,968 nipype.workflow INFO:
	 [Node] Setting-up "msktSNR.mask-t" in "/tmp/tmpj6qg0ina/msktSNR/_TR_1920_sub_03/mask-t".
210704-08:54:29,980 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t0" in "/tmp/tmpj6qg0ina/msktSNR/_TR_1920_sub_03/mask-t/mapflow/_mask-t0".
210704-08:54:29,985 nipype.workflow INFO:
	 [Node] Running "_mask-t0" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR

210704-08:54:32,942 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t13" in "/tmp/tmpj6qg0ina/msktSNR/_TR_1920_sub_03/mask-t/mapflow/_mask-t13".
210704-08:54:32,946 nipype.workflow INFO:
	 [Node] Running "_mask-t13" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr1/tsnr.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/MASKS/rPut_trans.nii.gz rPut_tsnr_ses-03.nii.gz
210704-08:54:33,155 nipype.workflow INFO:
	 [Node] Finished "_mask-t13".
210704-08:54:33,158 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t14" in "/tmp/tmpj6qg0ina/msktSNR/_TR_1920_sub_03/mask-t/mapflow/_mask-t14".
210704-08:54:33,163 nipype.workflow INFO:
	 [Node] Running "_mask-t14" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/tSNR/sub-03/TR1920/SNR/_do-tsnr0/tsnr.nii.gz -nan -mul /scrat